# Using DeepEval for AWS Bedrock LLM evaluation

## Build an agent with AWS Strands Agent

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. We will quick create the Strands agent first.

**Prerequisites**

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Basic understanding of Python programming

Lets get started !

In [1]:
%pip install strands-agents strands-agents-tools boto3 botocore -Uqqq

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Restart kernel (only works on Linux)
import os

os._exit(00)

**Optional -- Set Your AWS Credentials** 

There are multiple ways to set your AWS Credentials depending on your environment.

In [ ]:
session = boto3.Session(
    region_name="us-east-1",
    aws_access_key_id="<YOUR_ACCESS_KEY_ID>",
    aws_secret_access_key="<YOUR_SECRET_ACCESS_KEY>",
    aws_session_token="<YOUR_SESSION_TOKEN>",
)

**Custom Tool Demonstration**

The strands-agents-tools repository provides some in-built tools which you can import. You can also create custom tools using the @tool decorator. We can create agents with built-in and custom tools. For instance, adding the built-in tool of a word_count

In [1]:
from strands import Agent, tool
from strands_tools import calculator, current_time, python_repl,file_read,shell,file_write
import nest_asyncio

# Apply nest_asyncio at the start
nest_asyncio.apply()

@tool
def word_count(text: str) -> int:
    """Count words in text.

    This docstring is used by the LLM to understand the tool's purpose.
    """
    return len(text.split())


**Define an agent**

This will create an assistant agent with the default model provider, Amazon Bedrock, and set the model to Claude 3.7 Sonnet, in the region of your AWS setup. 

We set the BYPASS_TOOL_CONSENT environment parameter to true to make sure agent can automatically use tools without approval.

In [2]:
import os
os.environ["BYPASS_TOOL_CONSENT"] = "true"

Then, we define the agent and the tools.

In [3]:
agent = Agent(tools=[calculator, current_time, python_repl, word_count,file_read,shell,file_write],model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")

We use a complex task to test our assistance agent. The Prompt is: 

"clone this repo and get it to generate an image locally https://github.com/ChenDarYen/NitroFusion"

We can see the process:

* First, agent use Tool #1: shell to clone the repository
* Then, agent examine the repository to understand how to use it, it use Tool #2: file_read to read readme.md file.
* Next, it checks the requirements by using Tool #3: file_read to read requirement.txt and setup for running the model.
* Agent then create a script to use this model for image generation. it uses Tool #4: file_write to write scripts.
* Next, agent thinks that it should run script to generate an image, so it uses the Tool #6: shell to generate and run command.
* When the model has successfully generated an image using NitroFusion with a single step, agent use Tool #7: shell to verify the image was created.
* Agent trys another example, this time with the vibrant model and 2 inference steps and use Tool #8: shell
* Finally, it use Tool #9: shell to verify both images were created.

You can see this is a complex task. The agent needs to git clone the repo, install the environment, understand the code, generate the right scripts for image generation, download and run the code, verify images are generated successfully and provide a completion summary. Agent has 9 tool use calls in this task, choose 3 different but right tools within 7 tools configuration.

You can run the cell below to see the process.



In [4]:
message = "You are a helpful assistant that provides concise responses. clone this repo and get it to generate an image locally https://github.com/ChenDarYen/NitroFusion"

results = agent(message)

I'll help you clone the NitroFusion repository and get it to generate an image locally. Let's do this step by step.

First, I'll clone the repository:
Tool #1: shell


Command Execution Plan

╭───────────────────────────────────────── 🚀 Command Execution Preview ──────────────────────────────────────────╮
│                                                                                                                 │
│         🔷 Command   git clone https://github.com/ChenDarYen/NitroFusion                                        │
│     📁 Working Dir   /home/ec2-user/SageMaker/deepeval                                                          │
│   ⚡ Parallel Mode   ✗ No                                                                                       │
│    🛡️ Ignore Errors   ✗ No                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

⏳ Starting Command Execution...

Cloning into 'NitroFusion'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 9), reused 8 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 1.67 MiB | 51.93 MiB/s, done.
Resolving deltas: 100% (9/9), done.


...

-rw-rw-r-- 1 ec2-user ec2-user 1867021 Jun 10 03:20 NitroFusion/fantasy_forest.png
-rw-rw-r-- 1 ec2-user ec2-user 1425583 Jun 10 03:18 NitroFusion/landscape_output.png


✅ Command Execution Complete

╭───────────────────────────────────────────── ✅ Execution Summary ──────────────────────────────────────────────╮
│                                                                                                                 │
│   Total Commands   1                                                                                            │
│       Successful   1                                                                                            │
│           Failed   0                                                                                            │
│   Execution Mode   Sequential                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🟢 Command Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│      Status   ✓ Success                                                                                         │
│   Exit Code   0                                                                                                 │
│     Command   ls -la NitroFusion/landscape_output.png NitroFusion/fantasy_forest.png                            │
│      Output   -rw-rw-r-- 1 ec2-user ec2-user 1867021 Jun 10 03:20 NitroFusion/fantasy_forest.png                │
│               -rw-rw-r-- 1 ec2-user ec2-user 1425583 Jun 10 03:18 NitroFusion/landscape_output.png              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Summary

I've successfully helped you clone the NitroFusion repository and set it up to generate images locally. Here's what we accomplished:

1. Cloned the NitroFusion repository from GitHub
2. Created a Python script (`generate_image.py`) that implements the model's custom scheduler and provides an easy-to-use interface for generating images
3. Successfully generated two images:
   - A landscape image using the realism model with 1 inference step
   - A fantasy forest image using the vibrant model with 2 inference steps

The script we created can be used with the following options:
- `--prompt`: Text prompt for image generation
- `--model`: Choose between "realism" or "vibrant" models
- `--steps`: Number of inference steps (1-4)
- `--guidance`: Guidance scale (default: 0)
- `--output`: Output image path
- `--device`: Device to use ("cuda" or "cpu")

You can now use this script to generate more images by running commands like:
```
python generate_image.py --prompt "your prompt here

**See the excution and tool use results**

In [5]:
for m in agent.messages:
    for content in m["content"]:
        if "toolUse" in content:
            print("Tool Use:")
            tool_use = content["toolUse"]
            print("\tToolUseId: ", tool_use["toolUseId"])
            print("\tname: ", tool_use["name"])
            print("\tinput: ", tool_use["input"])
        if "toolResult" in content:
            print("Tool Result:")
            tool_result = m["content"][0]["toolResult"]
            print("\tToolUseId: ", tool_result["toolUseId"])
            print("\tStatus: ", tool_result["status"])
            print("\tContent: ", tool_result["content"])
            print("=======================")

Tool Use:
	ToolUseId:  tooluse_UA4OWXqnRWqQ37VteKRilQ
	name:  shell
	input:  {'command': 'git clone https://github.com/ChenDarYen/NitroFusion'}
Tool Result:
	ToolUseId:  tooluse_UA4OWXqnRWqQ37VteKRilQ
	Status:  success
	Content:  [{'text': 'Execution Summary:\nTotal commands: 1\nSuccessful: 1\nFailed: 0'}, {'text': "Command: git clone https://github.com/ChenDarYen/NitroFusion\nStatus: success\nExit Code: 0\nOutput: Cloning into 'NitroFusion'...\r\nremote: Enumerating objects: 22, done.\x1b[K\r\nremote: Counting objects:   4% (1/22)\x1b[K\rremote: Counting objects:   9% (2/22)\x1b[K\rremote: Counting objects:  13% (3/22)\x1b[K\rremote: Counting objects:  18% (4/22)\x1b[K\rremote: Counting objects:  22% (5/22)\x1b[K\rremote: Counting objects:  27% (6/22)\x1b[K\rremote: Counting objects:  31% (7/22)\x1b[K\rremote: Counting objects:  36% (8/22)\x1b[K\rremote: Counting objects:  40% (9/22)\x1b[K\rremote: Counting objects:  45% (10/22)\x1b[K\rremote: Counting objects:  50% (11/22)\x1b[K\rrem

## Achieve component level agent evaluation

We have built an complex agent just now. To evaluate this agent performance with component-level, we can separately evaluate there core characteristics. For example: 

* Evaluating Tool-Use
* Evaluating Agentic Workflows
* Agentic Reasoning Evaluation
* RAG Metrics



### Use DeepEval for tool use evaluation

First we use DeepEval for tool use evaluation.

**Tool correctness**


Tool Correctness assesses whether an agent’s tool-calling behavior aligns with expectations by verifying that all required tools were correctly called. Unlike most LLM evaluation metrics, the Tool Correctness metric is a deterministic measure and not an LLM-judge.

The tool correctness evaluation in tool_correctness.py works through three main modes:

1. Exact Match Mode (should_exact_match=True):

- Requires exact match of tool names, order, and optionally parameters/outputs
- Score is binary: 1.0 for perfect match, 0.0 otherwise

2. Order-Aware Mode (should_consider_ordering=True):

- Uses weighted Longest Common Subsequence algorithm
- Considers tool order while allowing partial matches
- Score = weighted_length / expected_tools_length

3. Default Mode (non-exact match):

- Ignores order, finds best matches for each expected tool
- Calculates partial scores for parameter matches
- Score = total_match_score / expected_tools_count

The evaluation also provides detailed reasons for the score, including missing tools, order issues, and parameter mismatches. The threshold parameter (default 0.5) determines pass/fail, with strict_mode requiring perfect scores.



In [6]:
from typing import Dict, List, Optional, Union, Any
from deepeval.models.llms.amazon_bedrock_model import AmazonBedrockModel
import nest_asyncio

# Apply nest_asyncio at the start
nest_asyncio.apply()

# Initialize the Bedrock model (e.g., Claude)
model = AmazonBedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name="us-east-1"
)


In [7]:

from deepeval import evaluate
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ToolCorrectnessMetric
from deepeval.test_case import LLMTestCase, ToolCall,ToolCallParams
from deepeval import evaluate
from extract_tool_calls import extract_tool_calls_from_strands

我们写了一个函数extract_tool_calls_from_strands(),来从 agent 的返回结果中，提取tool use 的信息，并构建成Deepeval 所需要的结构类型。

In [8]:

tool_calls, final_text = extract_tool_calls_from_strands(agent.messages)

if not final_text:
    print("Warning: No text found in messages, falling back to string conversion")
    try:
        final_text = str(results.response)
    except:
        final_text = str(results)

# Debug tool calls
print("\nDebug - Tool calls:", tool_calls)

# Create test case with string output
test_case = LLMTestCase(
    input=message,
    actual_output=final_text,
    tools_called=tool_calls,
    expected_tools=[ToolCall(name="file_read"),ToolCall(name="file_write"),ToolCall(name="shell")]
    
)

task_Correctness_metric = ToolCorrectnessMetric(model=model)

# Run evaluation synchronously
evaluate(
    test_cases=[test_case],
    metrics=[task_Correctness_metric],
)


Created ToolCall: {'name': 'shell', 'description': 'Tool used in the conversation: shell', 'reasoning': None, 'output': ["{'text': 'Execution Summary:\\nTotal commands: 1\\nSuccessful: 1\\nFailed: 0'}", '{\'text\': "Command: git clone https://github.com/ChenDarYen/NitroFusion\\nStatus: success\\nExit Code: 0\\nOutput: Cloning into \'NitroFusion\'...\\r\\nremote: Enumerating objects: 22, done.\\x1b[K\\r\\nremote: Counting objects:   4% (1/22)\\x1b[K\\rremote: Counting objects:   9% (2/22)\\x1b[K\\rremote: Counting objects:  13% (3/22)\\x1b[K\\rremote: Counting objects:  18% (4/22)\\x1b[K\\rremote: Counting objects:  22% (5/22)\\x1b[K\\rremote: Counting objects:  27% (6/22)\\x1b[K\\rremote: Counting objects:  31% (7/22)\\x1b[K\\rremote: Counting objects:  36% (8/22)\\x1b[K\\rremote: Counting objects:  40% (9/22)\\x1b[K\\rremote: Counting objects:  45% (10/22)\\x1b[K\\rremote: Counting objects:  50% (11/22)\\x1b[K\\rremote: Counting objects:  54% (12/22)\\x1b[K\\rremote: Counting objects:

✨ You're running DeepEval's latest Tool Correctness Metric! (using None, strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:00, 97.06test case/s]



Metrics Summary

  - ✅ Tool Correctness (score: 1.0, threshold: 0.5, strict: False, evaluation model: None, reason: All expected tools ['file_read', 'file_write', 'shell'] were called (order not considered)., error: None)

For test case:

  - input: You are a helpful assistant that provides concise responses. clone this repo and get it to generate an image locally https://github.com/ChenDarYen/NitroFusion
  - actual output: {'text': '## Summary\n\nI\'ve successfully helped you clone the NitroFusion repository and set it up to generate images locally. Here\'s what we accomplished:\n\n1. Cloned the NitroFusion repository from GitHub\n2. Created a Python script (`generate_image.py`) that implements the model\'s custom scheduler and provides an easy-to-use interface for generating images\n3. Successfully generated two images:\n   - A landscape image using the realism model with 1 inference step\n   - A fantasy forest image using the vibrant model with 2 inference steps\n\nThe script we c

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Tool Correctness', threshold=0.5, success=True, score=1.0, reason="All expected tools ['file_read', 'file_write', 'shell'] were called (order not considered).", strict_mode=False, evaluation_model=None, error=None, evaluation_cost=None, verbose_logs='Expected Tools:\n[\n    ToolCall(\n        name="file_read"\n    ),\n    ToolCall(\n        name="file_write"\n    ),\n    ToolCall(\n        name="shell"\n    )\n] \n \nTools Called:\n[\n    ToolCall(\n        name="shell",\n        description="Tool used in the conversation: shell",\n        input_parameters={\n            "command": "git clone https://github.com/ChenDarYen/NitroFusion"\n        },\n        output=["{\'text\': \'Execution Summary:\\\\nTotal commands: 1\\\\nSuccessful: 1\\\\nFailed: 0\'}", \'{\\\'text\\\': "Command: git clone https://github.com/ChenDarYen/NitroFusion\\\\nStatus: success\\\\nExit Code: 0\\\\nOutput: C

**Tool Efficiency**


Equally important to tool correctness is tool efficiency. Inefficient tool-calling patterns can increase response times, frustrate users, and significantly raise operational costs.


Let’s explore how tool efficiency can be evaluated:

* Redundant Tool --  Usage measures how many tools are invoked unnecessarily — those that do not directly contribute to achieving the intended outcome. This can be calculated as the percentage of unnecessary tools relative to the total number of tool invocations.
* Tool Frequency -- evaluates whether tools are being called more often than necessary. This method penalizes tools that exceed a predefined threshold for the number of calls required to complete a task (many times this is just 1).

In [9]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams


We use G-eval for tool efficiency evaluation. We define the task and criteria:

- task_name="Tool Efficiency",
- criteria="""Determine whether the tool effectively be used.Redundant Tool Usage measures how many tools are invoked unnecessarily — those that do not directly contribute to achieving the intended outcome.Tool Frequency evaluates whether tools are being called more often than necessary."""




In [12]:


# Create test case
test_case = LLMTestCase(
    input=message,
    actual_output=final_text,
    tools_called=tool_calls
)


# Create G-Eval metric for tool efficiency with Bedrock model
g_eval_metric = GEval(
    name="Tool Efficiency",
    criteria="""
Determine whether the tool effectively be used.
Redundant Tool Usage measures how many tools are invoked unnecessarily — those that do not directly contribute to achieving the intended outcome.
Tool Frequency evaluates whether tools are being called more often than necessary.
""",
    threshold=0.7,  # Set a reasonable threshold for tool efficiency
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.TOOLS_CALLED
    ],
    model=model  # Explicitly pass the Bedrock model
)

# Run evaluation synchronously
evaluate(
    test_cases=[test_case],
    metrics=[g_eval_metric],
)


✨ You're running DeepEval's latest Tool Efficiency (GEval) Metric! (using 
us.anthropic.claude-3-7-sonnet-20250219-v1:0, strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:28, 28.99s/test case]



Metrics Summary

  - ✅ Tool Efficiency (GEval) (score: 1.0, threshold: 0.7, strict: False, evaluation model: us.anthropic.claude-3-7-sonnet-20250219-v1:0, reason: The assistant effectively accomplished the task of cloning and setting up the NitroFusion repository for local image generation. The tool usage was optimal and efficient: first cloning the repo, then reading the README to understand requirements, creating a Python script implementing the model's custom scheduler, installing dependencies, and successfully generating two test images with different models and parameters. Each tool call directly contributed to solving the task, with no unnecessary repetition or redundant information gathering. The sequence of tool calls followed a logical progression from setup to execution, and the final output provided a clear summary of what was accomplished along with instructions for future use., error: None)

For test case:

  - input: You are a helpful assistant that provides concise res

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Tool Efficiency (GEval)', threshold=0.7, success=True, score=1.0, reason="The assistant effectively accomplished the task of cloning and setting up the NitroFusion repository for local image generation. The tool usage was optimal and efficient: first cloning the repo, then reading the README to understand requirements, creating a Python script implementing the model's custom scheduler, installing dependencies, and successfully generating two test images with different models and parameters. Each tool call directly contributed to solving the task, with no unnecessary repetition or redundant information gathering. The sequence of tool calls followed a logical progression from setup to execution, and the final output provided a clear summary of what was accomplished along with instructions for future use.", strict_mode=False, evaluation_model='us.anthropic.claude-3-7-sonnet-20250219-

**Evaluation results**

First, DeepEval evaluate the Tool Efficiency score is 1.0. The reason is:

The assistant effectively accomplished the task of cloning and setting up the NitroFusion repository for local image generation. The tool usage was optimal and efficient: first cloning the repo, then reading the README to understand requirements, creating a Python script implementing the model's custom scheduler, installing dependencies, and successfully generating two test images with different models and parameters. Each tool call directly contributed to solving the task, with no unnecessary repetition or redundant information gathering. The sequence of tool calls followed a logical progression from setup to execution, and the final output provided a clear summary of what was accomplished along with instructions for future use., error: None)

原因：该助手高效地完成了克隆和设置 NitroFusion 仓库以生成本地镜像的任务。工具的使用非常高效：首先克隆仓库，然后阅读 README 文件以了解需求，创建实现模型自定义调度程序的 Python 脚本，安装依赖项，并成功生成两个包含不同模型和参数的测试镜像。每次工具调用都直接有助于解决任务，无需不必要的重复或冗余信息收集。工具调用顺序遵循从设置到执行的逻辑顺序，最终输出清晰地概述了所完成的工作以及未来使用的说明。错误：无




The evaluation step that model generate is :

Evaluation Steps:
* "Check if the Input clearly defines a task that requires tool usage, and verify if the Actual Output accomplishes this task."
* "Examine Tools Called to identify any tools that were invoked but whose outputs were not utilized in the Actual Output or did not contribute to solving the task in the Input."
* "Compare the number of tool calls against the minimum necessary to complete the task, noting instances where the same tool was called multiple times when a single call would suffice."
* "Assess whether the sequence of tool calls was efficient, with no unnecessary repetition or redundant information gathering that could have been consolidated."

评估步骤：
* “检查输入是否明确定义了需要使用工具的任务，并验证实际输出是否完成了此任务。”
* “检查调用的工具，以识别任何已调用但其输出未在实际输出中使用或对解决输入中的任务无贡献的工具。”
* “将工具调用次数与完成任务所需的最小调用次数进行比较，并注意同一工具被多次调用的情况，而一次调用即可完成。”
* “评估工具调用顺序是否高效，没有不必要的重复或冗余信息收集，这些信息本可以合并。”



## Evaluating Agentic Workflows

**Task Completion**

A critical metric for assessing agent workflows is Task Completion (also known as task success or goal accuracy). This metric measures how effectively an LLM agent completes a user-given task. 

However, in real-world applications, agents are often required to perform a diverse set of tasks—many of which may lack predefined ground-truth datasets.DeepEval’s Task Completion metric addresses these challenges by leveraging LLMs to:

* Determine the task from the user’s input.
* Analyze the reasoning steps, tool usage, and final response to assess whether the task was successfully completed.

So we don't need to provide groundtruth for evaluation.

In [11]:
from deepeval.metrics import TaskCompletionMetric

# Create test case
test_case = LLMTestCase(
    input=message,
    actual_output=final_text,
    tools_called=tool_calls
)


task_completion_metric = TaskCompletionMetric(model=model)

# Run evaluation synchronously
evaluate(
    test_cases=[test_case],
    metrics=[task_completion_metric],
)

✨ You're running DeepEval's latest Task Completion Metric! (using us.anthropic.claude-3-7-sonnet-20250219-v1:0, 
strict=False, async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:29, 29.96s/test case]



Metrics Summary

  - ✅ Task Completion (score: 0.95, threshold: 0.5, strict: False, evaluation model: us.anthropic.claude-3-7-sonnet-20250219-v1:0, reason: The actual outcome successfully achieved the core goal of cloning the NitroFusion repository and setting it up to generate images locally, as evidenced by the successful generation of two different images using different models and parameters. The system demonstrated full functionality of the setup by creating a working Python script and producing actual image outputs. The only reason for not giving a perfect score is that we don't have confirmation that all possible repository features were set up, but the primary goal was clearly accomplished., error: None)

For test case:

  - input: You are a helpful assistant that provides concise responses. clone this repo and get it to generate an image locally https://github.com/ChenDarYen/NitroFusion
  - actual output: {'text': '## Summary\n\nI\'ve successfully helped you clone the NitroF

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Task Completion', threshold=0.5, success=True, score=0.95, reason="The actual outcome successfully achieved the core goal of cloning the NitroFusion repository and setting it up to generate images locally, as evidenced by the successful generation of two different images using different models and parameters. The system demonstrated full functionality of the setup by creating a working Python script and producing actual image outputs. The only reason for not giving a perfect score is that we don't have confirmation that all possible repository features were set up, but the primary goal was clearly accomplished.", strict_mode=False, evaluation_model='us.anthropic.claude-3-7-sonnet-20250219-v1:0', error=None, evaluation_cost=0.0, verbose_logs='User Goal: Clone the NitroFusion repository and set it up to generate images locally. \n \nTask Outcome: The system cloned the NitroFusion re

**Evaluation results**

First, DeepEval evaluate the Task Completion score is 0.95. The reason is:

The actual outcome successfully achieved the core goal of cloning the NitroFusion repository and setting it up to generate images locally, as evidenced by the successful generation of two different images using different models and parameters. The system demonstrated full functionality of the setup by creating a working Python script and producing actual image outputs. The only reason for not giving a perfect score is that we don't have confirmation that all possible repository features were set up, but the primary goal was clearly accomplished., error: None)

实际结果成功实现了克隆 NitroFusion 存储库并将其设置为本地生成图像的核心目标，这一点从使用不同模型和参数成功生成两张不同的图像可以看出。该系统通过创建可运行的 Python 脚本并生成实际图像输出，演示了设置的全部功能。未能给出满分的唯一原因是我们无法确认所有可能的存储库功能都已设置完毕，但主要目标显然已经达成。（错误：无）

The score and reason are make scence.



The task goal extracted by LLM is:
- Goal: Clone the NitroFusion repository and set it up to generate images locally. 

The task Outcome summarized by LLM is:
- Task Outcome: The system cloned the NitroFusion repository, created a Python script for image generation, and successfully generated two images: a landscape using the realism model with 1 inference step and a fantasy forest using the vibrant model with 2 inference steps.

and the task summary is:
```
Summary

I've successfully helped you clone the NitroFusion repository and set it up to generate images locally. Here's what we accomplished:

1. Cloned the NitroFusion repository from GitHub
2. Created a Python script (`generate_image.py`) that implements the model's custom scheduler and provides an easy-to-use interface for generating images
3. Successfully generated two images:
   - A landscape image using the realism model with 1 inference step
   - A fantasy forest image using the vibrant model with 2 inference steps

The script we created can be used with the following options:
- `--prompt`: Text prompt for image generation
- `--model`: Choose between "realism" or "vibrant" models
- `--steps`: Number of inference steps (1-4)
- `--guidance`: Guidance scale (default: 0)
- `--output`: Output image path
- `--device`: Device to use ("cuda" or "cpu")

You can now use this script to generate more images by running commands like:

python generate_image.py --prompt "your prompt here" --model realism --steps 1 --output output.png
```

Deepeval use Goal, summurized Task Outcome and output for task completion evaluation.

## Next
In this lab, we use the AWS open-source solution called Strands agents to build an assistant agent and complete a complex task. Then we use deepeval for component-level agent evaluation, including tool correctness, tool efficiency, and task completion.

Because we use one of the best models, Claude 3.7, we complete the task successfully and get positive evaluation results. You can change the model to Nova or Hiku models to test again and see the evaluation performance.